In [1]:
from matplotlib import pyplot as plt
import time
import datetime
from utils import *
from data_processing import *
today = datetime.date.today()


In [2]:
df = read_cmapss(TsConf.input_url)
feat_names = df.columns.values[3:-1]
target_name = df.columns.values[-1]
df[feat_names] = data_norm(df[feat_names])

In [3]:
select_feat = ["Total temp at HPC out (T30)", 
               "Total temp at LPT out (T50)", 
               "Physical core speed (Nc)", 
               "Static pres at HPC out (Ps30)", 
               "Corrected core speed (NRc)", 
               "Bypass Ratio (BPR)", 
               "Bleed Enthalpy (htBleed)"]

In [4]:
train = df[(["FILEID","ENGINEID", "TIMECYCLE"]+select_feat+["RUL"])][df.FILEID.isin(TsConf.train_fids)]
test = df[(["FILEID","ENGINEID", "TIMECYCLE"]+select_feat+["RUL"])][df.FILEID.isin(TsConf.test_fids)]

In [5]:
train.tail()

,FILEID,ENGINEID,TIMECYCLE,Total temp at HPC out (T30),Total temp at LPT out (T50),Physical core speed (Nc),Static pres at HPC out (Ps30),Corrected core speed (NRc),Bypass Ratio (BPR),Bleed Enthalpy (htBleed),RUL
20626,101,100,196,1.116880,1.242910,1.043518,1.141351,0.629226,-0.740904,1.179266,4
20627,101,100,197,1.172109,1.279284,1.042422,1.132578,0.615475,-0.716515,1.114725,3
20628,101,100,198,1.154829,1.239603,1.044534,1.147200,0.672353,-0.648944,1.211537,2
20629,101,100,199,1.178547,1.227479,1.065438,1.234927,0.650352,-0.683196,1.114725,1
20630,101,100,200,1.137210,1.268703,1.032719,1.179366,0.625850,-0.730242,1.146996,0


# Model training

## LSTM (Sibur)

In [6]:
train = TsDataFrame(train)
test = TsDataFrame(test)
n_eng_train = max(train.ENGINEID)
n_eng_test = max(test.ENGINEID)

In [7]:
train.target = TsConf.target
train.groupids = TsConf.groupids
train.timestamp = TsConf.timestamp
test.target = TsConf.target
test.groupids = TsConf.groupids
test.timestamp = TsConf.timestamp

In [8]:
    config = TsConf(train.columns)
    Train = True
    Predict = True
    mode = "Train"

In [9]:
        X = tf.placeholder(tf.float32, [None, config.sequence_length, config.n_channels], name='inputs')
        Y = tf.placeholder(tf.float32, [None, config.sequence_length], name='labels')
        keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        learning_rate_ = tf.placeholder(tf.float32, name='learning_rate')
        is_train = tf.placeholder(dtype=tf.bool, shape=None, name="is_train")

        conv_last_layer = X

        shape = conv_last_layer.get_shape().as_list()
        print('My Conv Shape:',shape)
        CNN_flat = tf.reshape(conv_last_layer, [-1, shape[1] * shape[2]])

        dence_layer_1 = dense_layer(CNN_flat, size=config.sequence_length * config.n_channels, activation_fn=tf.nn.relu, batch_norm=False,
                                    phase=is_train, drop_out=True, keep_prob=keep_prob,
                                    scope="fc_1")
        lstm_input = tf.reshape(dence_layer_1, [-1, config.sequence_length, config.n_channels])

        cell = get_RNNCell(['LSTM'] * config.num_layers, keep_prob=keep_prob, state_size=config.lstm_size)
        init_states = cell.zero_state(config.batch_size, tf.float32)

        # For each layer, get the initial state. states will be a tuple of LSTMStateTuples.
        states = get_state_variables(config.batch_size, cell)

        # Unroll the LSTM
        rnn_output, new_states = tf.nn.dynamic_rnn(cell, lstm_input, dtype=tf.float32, initial_state=states)

        # Add an operation to update the train states with the last state tensors.
        update_op = get_state_update_op(states, new_states)
        reset_op = get_state_update_op(states, init_states)

        stacked_rnn_output = tf.reshape(rnn_output, [-1, config.lstm_size])  # change the form into a tensor

        dence_layer_2 = dense_layer(stacked_rnn_output, size=config.ann_hidden, activation_fn=tf.nn.relu, batch_norm=False,
                                    phase=is_train, drop_out=True, keep_prob=keep_prob,
                                    scope="fc_2")

        dence_layer_3 = dense_layer(dence_layer_2, size=config.ann_hidden, activation_fn=tf.nn.relu, batch_norm=False,
                                    phase=is_train, drop_out=True, keep_prob=keep_prob,
                                    scope="fc_2_2")

        output = dense_layer(dence_layer_3, size=1, activation_fn=None, batch_norm=False, phase=is_train, drop_out=False,
                             keep_prob=keep_prob,
                             scope="fc_3_output")

        prediction = tf.reshape(output, [-1])
        y_flat = tf.reshape(Y, [-1])

        h = prediction - y_flat

        tv = tf.trainable_variables()
        regularization_cost = tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv ])

        cost_function = tf.reduce_sum(tf.square(h)) + config.alpha*regularization_cost
        RMSE = tf.sqrt(tf.reduce_mean(tf.square(h)))
        optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost_function)

        saver = tf.train.Saver()


        if mode=="Train": model_summary(learning_rate=config.learning_rate, batch_size=config.batch_size, lstm_layers=config.num_layers,
                                lstm_layer_size=config.lstm_size, fc_layer_size=config.ann_hidden, sequence_length=config.sequence_length,
                                n_channels=config.n_channels, path_checkpoint=config.path_checkpoint, spacial_note='')


My Conv Shape: [None, 30, 7]


In [ ]:
    with tf.Session() as session:
        tf.global_variables_initializer().run()

        if Train == True:
            saver.restore(session, config.path_checkpoint)
            print("Model restored from file: %s" % config.path_checkpoint)

            cost = []
            plot_x = []
            plot_y1 = []
            plot_y2 = []
            iter_train = int(train.shape[0]/config.shift)
            iter_test = int(test.shape[0]/config.shift)
            print("Training set MSE")
            print("No epoches: ", config.epochs, "No itr: ", iter_train)
            __start = time.time()
            for ep in range(config.epochs):
                session.run(reset_op)
                training_generator = train.endless_batch(config.sequence_length, config.shift, config.batch_size)
                testing_generator = test.endless_batch(config.sequence_length, config.shift, config.batch_size)
                
                h1 = []
                t1 = []
                engine_id = 1
           
                while engine_id < n_eng_train:
                    ## training ##
                    train_gen = next(training_generator)
                    batch_x, batch_y = train_gen[1], train_gen[2]                   
                    session.run([optimizer, update_op],
                                feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.7, learning_rate_: config.learning_rate})
                    h_i = h.eval(feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0, learning_rate_: config.learning_rate})
                    cost.append(np.square(h_i))
                    h1.append(h_i)
                    
                    if batch_y[0,-1] == 0: #"RUL==0 end of current timeseries"
                        session.run(reset_op)
                        engine_id = train_gen[0][0,0,1]
                        print ("eng_id: ",engine_id, "  RMSE train:", np.sqrt(np.mean(np.square(h1))))

                rmse_train = np.sqrt(np.mean(np.square(h1)))

                save_path = saver.save(session, config.path_checkpoint)
                if os.path.exists(config.path_checkpoint + '.meta'):
                    print("Model saved to file: %s" % config.path_checkpoint)
                else:
                    print("NOT SAVED!!!", config.path_checkpoint)
                
                y_pred = []
                
                try:
                    while True:
                        test_gen = next(testing_generator)
                        x_test_batch, y_test_batch = test_gen[1], test_gen[2]
                        h_i, u = session.run([h, update_op], feed_dict={X: x_test_batch, Y: y_test_batch, keep_prob: 1.0, 
                                                                        learning_rate_: config.learning_rate})
                        t1.append(h_i)
                except StopIteration:
                    pass
                
                rmse_test = np.sqrt(np.mean(np.square(t1)))

                plot_x.append(ep)
                plot_y1.append(rmse_train)
                plot_y2.append(rmse_test)

                time_per_ep = (time.time() - __start)
                time_remaining = ((config.epochs - ep) * time_per_ep) / 3600
                print("LSTM", "epoch:", ep, "RMSE-train:", rmse_train, "RMSE-test", rmse_test, "lr", config.learning_rate,
                      "\ttime/epoch:", round(time_per_ep, 2), "\ttime_remaining: ",
                      int(time_remaining), " hr:", round((time_remaining % 1) * 60, 1), " min", "\ttime_stamp: ",
                      datetime.datetime.now().strftime("%Y.%m.%d-%H:%M:%S"))
                __start = time.time()
                
                if ep % 50 == 0 and ep != 0: 
                    plt.plot(plot_x, plot_y1, 'bo', plot_x, plot_y2, 'go')
                    plt.show()
                        
                if ep % 100 == 0 and ep != 0: 
                    config.learning_rate = config.learning_rate / 2


            save_path = saver.save(session, config.path_checkpoint)
            if os.path.exists(path_checkpoint + '.meta'):
                print("Model saved to file: %s" % path_checkpoint)
            else:
                print("NOT SAVED!!!", path_checkpoint)
            plt.plot(plot_x, plot_y1, 'bo', plot_x, plot_y2, 'go')
            plt.show()
        else:
            saver.restore(session, config.path_checkpoint)
            print("Model restored from file: %s" % config.path_checkpoint)
            if Predict == True:
                print("Prediction for submit...")
                x_predict = Xs
                y_predict = np.zeros((Xs.shape[0],Xs.shape[1]))

                predict_generator = batch_generator(x_predict, y_predict, config.batch_size, config.sequence_length,
                                                       online=True, online_shift=config.shift)

                full_prediction = []

                iteration = int(x_predict.shape[0] / config.shift)
                #print("iteration: %i, predgen %s" % (iteration, predict_generator))
                print("#of validation points:", x_predict.shape[0], "#datapoints covers from minibatch:",
                      config.batch_size * config.sequence_length, "iterations/epoch", iteration)

                for itr in range(iteration):
                    x_validate_batch, y_validate_batch = next(predict_generator)
                    #print (itr)
                    __y_pred, u = session.run([output, update_op], feed_dict={X: x_validate_batch, Y: y_validate_batch, keep_prob: 1.0})
                    #session.run(update_op)
                    for i in range(config.batch_size):
                        full_prediction.append(__y_pred[i*config.sequence_length])
                    #print(__y_pred.shape)
                    
                full_prediction = np.array(full_prediction)
                full_prediction = full_prediction.ravel()
            


INFO:tensorflow:Restoring parameters from ./save/save_lstm/lstm_2_layers
Model restored from file: ./save/save_lstm/lstm_2_layers
Training set MSE
No epoches:  1000 No itr:  20631
eng_id:  1.0   RMSE train: 17.691248
eng_id:  2.0   RMSE train: 15.986529
eng_id:  3.0   RMSE train: 19.306723
eng_id:  4.0   RMSE train: 19.554493
eng_id:  5.0   RMSE train: 18.284733
eng_id:  6.0   RMSE train: 17.5273
eng_id:  7.0   RMSE train: 17.108736
eng_id:  8.0   RMSE train: 16.978357
eng_id:  9.0   RMSE train: 17.518353
eng_id:  10.0   RMSE train: 17.322283
eng_id:  11.0   RMSE train: 16.83658
eng_id:  12.0   RMSE train: 16.50655
eng_id:  13.0   RMSE train: 16.339832
eng_id:  14.0   RMSE train: 16.19517
eng_id:  15.0   RMSE train: 15.989662
eng_id:  16.0   RMSE train: 15.835707
eng_id:  17.0   RMSE train: 15.545993
eng_id:  18.0   RMSE train: 15.42515
eng_id:  19.0   RMSE train: 15.33781
eng_id:  20.0   RMSE train: 15.349435
eng_id:  21.0   RMSE train: 15.205851
eng_id:  22.0   RMSE train: 15.110872


In [ ]:
y_submit = full_prediction*std_y+mean_y
sub_file = pd.DataFrame()
sub_file["timestamp"] = submit_X.index[-2872:]
sub_file["target"] = y_submit[-2872:]
sub_file.to_csv('submit2.csv', index=False)

## Gradient Boosting Regression

### TS fresh dataset

In [ ]:
#add_lag_roll(test.iloc[:,:-2], feat_names, lag=lag, step=step, roll=0).head()

In [ ]:
#df.groupby(["FILEID", "ENGINEID"])

In [ ]:
#X=pd.DataFrame()
#y=list()

In [ ]:
#for name, group in df.groupby(["FILEID", "ENGINEID"]):
    #X = pd.concat([X, add_lag_roll(group[feat_names], feat_names, lag=lag, step=step)], axis=0)
    #y = y.append(group["RUL"][lag:])
    
    

In [ ]:
#X.tail(1).values